
### In this notebook, we demonstrate how to read and filter the ATUS Well Being Data. 


1. **Importing the necessary library**:

2. **Loading the necessary files**:
    - The file path is set to `"/Users/ruchiraray/Documents/UT Austin/Soc-HHR/lexiconwex2021.csv"`. Replace this path with the path to your CSV file.

3. **Creating a dictionary of activities**:
    - A dictionary (`activity_code_dict`) is created from the filtered DataFrame, where the keys are the '6-digit activity code' values, and the corresponding values are the 'Activity' name.

4. **Filtering the DataFrame**:
    - The DataFrame based on age, income, gender



## Data Loading


To start off we first load the dataset, these are the csv file we get from ATUS. We select ATUS Summary as it has time info per activity. We also load lexiconwex2021.csv to map the activity to the 6-digit activity code given by ATUS to get a activity dictionary. This dictionary will be used for further analysis in the project.


Replace 'file_path' with the path to the CSV file you want to read

In [15]:
import pandas as pd

# Replace 'your_file_path.csv' with the path to the CSV file you want to read
# file_path = '/Users/ruchiraray/Documents/UT Austin/Soc-HHR/BEHAVIOR1kData/survey_analysis/lexiconwex2022.csv'
file_path = "/Users/ruchiraray/Documents/UT Austin/Soc-HHR/lexiconwex2021.csv"

# Attempt to read the CSV file

df = pd.read_csv(file_path, header=0)
activity_code_df = df[df['6-digit activity code'].notna()][['6-digit activity code', 'Activity']]

activity_code_dict = pd.Series(activity_code_df['Activity'].values, index=activity_code_df['6-digit activity code']).to_dict()
print(len(activity_code_dict))

466


In [16]:
import pandas as pd
atus_summary_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atussum-2021/atussum_2021.csv', header=0)

## Filtering 

We now look into filtering our data by by Sex, Income Range, Age Range, Race, Number of Household Children, to create a susbset we want to work with. We do this similar to ATUS time. Using the filtered datafrae, we take the intersection of ATUS Well Being to get our needed subset.



The following code blocks perform essential data preprocessing tasks required for our analysis. Each block is responsible for a specific aspect of the data filtering and preparation process:

1. **Sex**: This code block maps gender labels to numerical values and filters the dataset based on the selected gender.
2. **Income**: This code block maps income codes to income ranges, classifies income levels, and filters the dataset based on income classification. We also see filtering based on on a specified income level.
3. **Race**: This code block maps race labels to numerical values and filters the dataset based on the selected race.
4. **Age Range**: This code block filters the dataset based on a specified age range.

These preprocessing steps are crucial for ensuring that the dataset is accurately filtered and categorized according to the criteria specified in the study.

Below, we explain each code block in detail, including its purpose, the variables involved, and the logic applied.



### 1. Sex Dictionary and Filtering

This code block maps gender labels to numerical values and filters the dataset based on the selected gender. 

- **Variables**:
  - `sex_dict`: A dictionary mapping gender labels ('male', 'female') to numerical values (1, 2).
  - `sex`: A variable that stores the selected gender. If `sex` is specified, the dataset is filtered based on this value.

- **Logic**:
  - Check if `sex` is specified.
  - If `sex` is specified, convert it to the corresponding numerical value using `sex_dict`.
  - Filter the DataFrame to include only rows where the gender matches the specified value.



In [ ]:
sex_dict = {
    'male': 1,
    'female': 2
}

sex=None

# Sex


if sex:
    sex_number = sex_dict[sex]
    atus_summary_df=atus_summary_df[atus_summary_df['TESEX'] == sex_number]



### 2. Income Mapping and Classification

This code block maps income codes to income ranges, classifies income levels, and filters the dataset based on income classification.

To start off, we load the ATUS rost data for Number of Household memeber information. We use this information alonh with the poverty line information of 2021 to define bounds of low , mid and high income.

Over here , we use income mapping to filter based on the bins set by ATUS, we use income class to filter by our set way of defining low, mid and high income. We also have a third income range, where we filter based on the range. It must be noted that for income range and income class, they are both based on income_mapping as these are the incoe information provided to us by ATUS.


#### Variables:
- `income_mapping`: A dictionary mapping income codes to income ranges.
- `income_class`: A variable that stores the selected income class ('high', 'mid', 'low').

#### Logic:
1. Load the necessary datasets (`atuscps_df`, `household_members_df`, `thresh_df`).
2. Merge datasets to get the number of household members for each respondent.
3. Define a function `classify_income` that classifies income based on the income code, number of members, and poverty thresholds.
4. Apply the classification function to create an 'Income Classification' column.
5. Filter the DataFrame based on the specified income class.


In [17]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atusrost-2021/atusrost_2021.csv')

# Group by 'TUCASEID' and count the unique 'TULINENO' entries for each 'TUCASEID'
household_counts = df.groupby('TUCASEID')['TULINENO'].nunique().reset_index()

# Rename columns to reflect the data
household_counts.columns = ['TUCASEID', 'Number of Household Members']

# Save the result to a new CSV file
# household_counts.to_csv('household_members_ATUS.csv', index=False)

In [ ]:
income_mapping = {
    1:[0,5000],
    2:[5000,7499],
    3:[7500,9999],
    4:[10000,12499],
    5:[12500,14999],
    6:[15000,19999],
    7:[20000,24999],
    8:[25000,29999],
    9:[30000,34999],
    10:[35000,39999],
    11:[40000,49999],
    12:[50000,59999],
    13:[60000,74999],
    14:[75000,99999],
    15:[100000,149999],
    16:[150000,9999999999]
}

income=None

income_class=None
#high low mid



# Income Claasification by family member threshold + poverty line 2021
if income_class:   
    # Load the data
    atuscps_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atuscps-2021/atuscps_2021.csv')
    household_members_df = household_counts
    # pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/BEHAVIOR1kData/survey_analysis/household_members_ATUS.csv')
    thresh_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/BEHAVIOR1kData/survey_analysis/thresholds.csv')

    # Merge atuscps_df with household_members_df to get the number of members for each household
    atuscps_df = atuscps_df.merge(household_members_df, on='TUCASEID')

    # Function to classify income based on HEFAMINC, number of members, and thresholds
    def classify_income(row, thresh_df):
        income_code = row['HEFAMINC']
        members = row['Members']
        
        # Map the income code to a range using income_mapping
        lower_limit, upper_limit = income_mapping[income_code]

        # Find the appropriate threshold for the number of members
        if members >= 9:  # Assuming last row in thresh_df is for 5 and above
            thresh_row = thresh_df.iloc[-1]
        else:
            thresh_row = thresh_df[thresh_df['Size'] == members].iloc[0]

        # # Classify income
        # if lower_limit==0:
        #     return 'low'
        # else:
        # Determine if it's mid based on the mapping and thresholds
        low_thresh = thresh_row['poverty']
        high_thresh = thresh_row['high']
        if upper_limit <= low_thresh:
            return 'low'
        elif lower_limit >= high_thresh:
            return 'high'
        else:
            return 'mid'

    # Apply the classification function
    atuscps_df['Income Classification'] = atuscps_df.apply(classify_income, args=(thresh_df,), axis=1)

    atus_summary_df = atus_summary_df.merge(atuscps_df[['TUCASEID', 'Income Classification']], on='TUCASEID', how='left')

    atus_summary_df=atus_summary_df[atus_summary_df['Income Classification'] == "high"]

# Income

if income:
    # Load the data
    atuscps_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atuscps-2021/atuscps_2021.csv')

    # Merge atussum_df with classified_atuscps_df to get HEFAMINC value
    atus_summary_df = atus_summary_df.merge(atuscps_df[['TUCASEID', 'HEFAMINC']], on='TUCASEID', how='left')

    # Filter rows where HEFAMINC is 2
    atus_summary_df = atus_summary_df[atus_summary_df['HEFAMINC'] == income]



# Income Range -custom 



### 3. Race Dictionaries and Filtering

This code block maps race labels to numerical values and filters the dataset based on the selected race.

#### Explanation:

#### Variables:
- `raceasain_dict` and `race_dict`: Dictionaries mapping race labels to numerical values.
- `race_name`: A variable that stores the selected race.

#### Logic:
1. Check if `race_name` is specified.
2. If `race_name` is specified, convert it to the corresponding numerical value using `race_dict`.
3. Filter the DataFrame to include only rows where the race matches the specified value.


In [ ]:



start_age = 0
end_age = 1000


raceasain_dict = {
    'Asian Indian': 1,
    'Chinese': 2,
    'Filipino': 3,
    'Japanese': 4,
    'Korean': 5,
    'Vietnamese': 6,
    'Other': 7
}

race_dict = {
    'White only': 1,
    'Black only': 2,
    'American Indian, Alaskan Native only': 3,
    'Asian only': 4,
    'Hawaiian/Pacific Islander only': 5,
    'White-Black': 6,
    'White-American Indian': 7,
    'White-Asian': 8,
    'White-Hawaiian': 9,
    'Black-American Indian': 10,
    'Black-Asian': 11,
    'Black-Hawaiian': 12,
    'American Indian-Asian': 13,
    'American Indian-Hawaiian': 14,
    'Asian-Hawaiian': 15,
    'White-Black-American Indian': 16,
    'White-Black-Asian': 17,
    'White-Black-Hawaiian': 18,
    'White-American Indian-Asian': 19,
    'White-American Indian-Hawaiian': 20,
    'White-Asian-Hawaiian': 21,
    'Black-American Indian-Asian': 22,
    'White-Black-American Indian-Asian': 23,
    'White-American Indian-Asian-Hawaiian': 24,
    'Other 3 race combinations': 25,
    'Other 4 and 5 race combinations': 26
}


race_name = None



# Race


if race_name:
    race_number = race_dict[race_name]
    atus_summary_df=atus_summary_df[atus_summary_df['PTDTRACE'] == race_number]




### 4. Age Range Filtering

This code block filters the dataset based on a specified age range.

#### Explanation:

#### Variables:
- `start_age`: The beginning of the age range to filter.
- `end_age`: The end of the age range to filter.

#### Logic:
1. Define the `start_age` and `end_age` variables to specify the age range.
2. Filter the DataFrame to include only rows where the age (`TEAGE`) is within the specified range.


In [ ]:
# Age Range
start_age = 0
# int(input("Enter the start age: "))
end_age = 1000
# int(input("Enter the end age: "))
atus_summary_df = atus_summary_df[(atus_summary_df['TEAGE'] >= start_age) & (atus_summary_df['TEAGE'] <= end_age)]



# Number of Household Children - no direct way

,TUCASEID,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,t181501,t181599,t181601,t181801,t189999,t500101,t500103,t500105,t500106,t500107
0,20210101210037,7.575483e+06,-1,60,2,39,1,2,2,1,...,0,0,0,0,0,75,0,0,0,0
1,20210101210081,8.737183e+06,10,30,2,42,1,2,1,1,...,0,0,0,0,60,150,0,0,0,0
2,20210101210099,3.023910e+07,-1,43,1,39,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
3,20210101210101,1.116551e+07,-1,23,1,43,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
4,20210101210109,6.562833e+06,-1,20,1,40,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9082,20211212212479,3.571728e+07,-1,56,1,43,1,2,1,2,...,0,0,0,0,0,0,0,0,0,0
9083,20211212212502,1.220601e+07,-1,69,2,44,1,2,1,5,...,0,0,0,0,0,0,0,0,0,0
9084,20211212212555,4.810168e+06,-1,57,1,43,1,2,1,1,...,0,0,0,0,0,0,0,0,0,0
9085,20211212212567,1.138100e+07,6,34,2,43,1,2,2,1,...,0,0,0,0,0,0,0,0,0,0


Thus, given our filtered atus summry file, we take the intersection of it with ATUS Well Being to get our needed filtered subset. To do this we merge the 2 dataframes using "TUCASEID"

In [19]:
import pandas as pd

# Load the datasets
atusact_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atusact-2021/atusact_2021.csv',header=0)
# atussum_df = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/atussum-2021/atussum_2021.csv')


# Merge the filtered atussum_df with atusact_df on TUCASEID
atusact_subset_df = pd.merge(atusact_df, atus_summary_df[['TUCASEID']], on='TUCASEID', how='inner')

wb = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/wbact-2021/wbact_2021.csv')

# Merge the datasets on TUCASEID and TUACTIVITY_N
merged_data = pd.merge(atusact_subset_df, wb, on=['TUCASEID', 'TUACTIVITY_N'])
merged_data

,TUCASEID,TUACTIVITY_N,TEWHERE,TRTCCTOT_LN,TRTCC_LN,TRTCOC_LN,TRTEC_LN,TRTHH_LN,TRTNOHH_LN,TRTOHH_LN,...,WUINTERACT,WUMEANING,WUPAIN,WUPNORD,WUSAD,WUSADORD,WUSTRESS,WUSTRORD,WUTIRED,WUTRDORD
0,20210301210024,2,1,0,-1,0,-1,-1,-1,-1,...,1,5,6,5,0,4,2,2,5,3
1,20210301210024,5,7,0,-1,0,-1,-1,-1,-1,...,1,4,5,5,1,4,5,2,3,3
2,20210301210024,7,4,0,-1,0,-1,-1,-1,-1,...,1,5,4,5,0,4,2,2,3,3
3,20210301210094,3,12,0,-1,0,-1,-1,-1,-1,...,1,3,0,5,0,2,1,1,0,3
4,20210301210094,4,2,0,-1,0,-1,-1,-1,-1,...,1,5,0,5,0,2,4,1,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20456,20211212212567,11,1,20,20,0,-1,20,-1,20,...,1,6,2,5,3,4,2,3,5,1
20457,20211212212567,15,1,0,0,0,-1,0,-1,0,...,1,1,1,5,2,4,3,3,3,1
20458,20211212212591,3,1,30,30,0,-1,30,-1,30,...,1,6,4,2,0,1,0,5,0,4
20459,20211212212591,5,1,0,0,0,-1,0,-1,0,...,1,6,4,2,0,1,0,5,0,4


Now we find the Mean and Variance in Happiness and Meaningfulness

In [36]:


filtered_data = merged_data[(merged_data['WUHAPPY'] >= 0) & (merged_data['WUMEANING'] >= 0)]

# Group by 'TRCODE' and calculate the mean for 'WUHAPPY' and 'WUMEANING'
mean_values = filtered_data.groupby('TRCODE')[['WUHAPPY', 'WUMEANING']].mean().reset_index()


activity_time = pd.read_csv('/Users/ruchiraray/Documents/UT Austin/Soc-HHR/BEHAVIOR1kData/survey_analysis/activity_time_2021_mid.csv')
# activity_time=atus_summary_df

# Ensure the TRCODE and Activity Code are of the same type, adjust as necessary
activity_time['Activity Code'] = activity_time['Activity Code'].astype(int)

# Merge the DataFrames on the appropriate columns
merged_data_with_activities = pd.merge(mean_values, activity_time, left_on='TRCODE', right_on='Activity Code')

# Select and rename the columns as needed
final_data = merged_data_with_activities[['TRCODE', 'WUHAPPY', 'WUMEANING', 'Activity']]

final_data.to_csv("wb_2021_.csv", index=False) 

In [37]:
import pandas as pd

# Load the datasets
activity_time = pd.read_csv('activity_time_2021_.csv')
wb = merged_data

# Ensure the TRCODE and Activity Code are of the same type
activity_time['Activity Code'] = activity_time['Activity Code'].astype(int)

# Filter out rows with negative WUHAPPY or WUMEANING
filtered_data = wb[(wb['WUHAPPY'] >= 0) & (wb['WUMEANING'] >= 0)]

# Group by 'TRCODE' and calculate the mean and variance for 'WUHAPPY' and 'WUMEANING'
mean_values = filtered_data.groupby('TRCODE')[['WUHAPPY', 'WUMEANING']].mean().reset_index()
variance_values = filtered_data.groupby('TRCODE')[['WUHAPPY', 'WUMEANING']].var().reset_index()

# Rename the variance columns to avoid confusion
variance_values.rename(columns={'WUHAPPY': 'var_WUHAPPY', 'WUMEANING': 'var_WUMEANING'}, inplace=True)

# Merge the mean and variance data
mean_variance_data = pd.merge(mean_values, variance_values, on='TRCODE')

# Merge the mean_variance_data with the activity_time data
merged_data_with_activities = pd.merge(mean_variance_data, activity_time, left_on='TRCODE', right_on='Activity Code')

# Select and rename the columns as needed
final_data = merged_data_with_activities[['TRCODE', 'WUHAPPY', 'WUMEANING', 'var_WUHAPPY', 'var_WUMEANING', 'Activity']]
final_data.rename(columns={'WUHAPPY': 'Mean_WUHAPPY', 'WUMEANING': 'Mean_WUMEANING'}, inplace=True)

# Save the final data to a CSV file
final_data.to_csv('wb_2021_.csv', index=False)

# Output the final dataframe for verification
print(final_data)


     TRCODE  Mean_WUHAPPY  Mean_WUMEANING  var_WUHAPPY  var_WUMEANING  \
0     10301      3.647541        4.049180     3.684663       4.262024   
1     10399      0.000000        0.000000          NaN            NaN   
2     20101      3.888889        3.884444     3.199208       3.768354   
3     20102      3.968652        3.742947     2.565052       3.977741   
4     20103      4.640000        4.840000     1.906667       2.306667   
..      ...           ...             ...          ...            ...   
270  181601      4.285714        4.142857     4.238095       2.476190   
271  181801      4.142857        2.857143     1.809524       3.476190   
272  189999      4.592593        3.888889     2.359189       5.345912   
273  500101      3.972727        4.172727     3.421268       4.107506   
274  500103      4.789474        5.105263     2.397661       1.321637   

                                      Activity  
0                     Health-related self care  
1                        

/var/folders/7b/w2c_xdts2nd4b407r_j90r9h0000gn/T/ipykernel_24459/3958928541.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data.rename(columns={'WUHAPPY': 'Mean_WUHAPPY', 'WUMEANING': 'Mean_WUMEANING'}, inplace=True)


In [34]:
variance_values

,TRCODE,var_WUHAPPY,var_WUMEANING
0,10301,3.684663,4.262024
1,10399,NaN,NaN
2,20101,3.199208,3.768354
3,20102,2.565052,3.977741
4,20103,1.906667,2.306667
...,...,...,...
274,181601,4.238095,2.476190
275,181801,1.809524,3.476190
276,189999,2.359189,5.345912
277,500101,3.421268,4.107506
